In [1]:
import sys
import os
sys.path.append("../wanpy")

# from WanPy import *
from pythtb import *
from wpythtb import *
import models
import plot 

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# import numba
import sympy as sp
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.sparse.linalg import eigsh
from numba import njit, prange

%load_ext line_profiler
%timeit

In [2]:
delta = 1
t = 1
t2 = -0.3

n_super_cell = 2
model = models.Haldane(delta, t, t2).make_supercell([[n_super_cell, 0], [0, n_super_cell]])

low_E_sites = np.arange(0, model.get_num_orbitals(), 2)
n_orb = model.get_num_orbitals()
n_occ = int(n_orb/2)

orbs = model.get_orb()
lat_vecs = model.get_lat() # lattice vectors

low_E_sites = np.arange(0, model.get_num_orbitals(), 2)
high_E_sites = np.arange(1, model.get_num_orbitals(), 2)

In [3]:
tf_list = [ [(orb, 1) ] for orb in low_E_sites ]

WFs = Wannier(model, [20, 20])
WFs.single_shot(tf_list)

In [14]:
%lprun -f WFs.subspace_selec WFs.subspace_selec(verbose=True, iter_num=100, tol=1e-15)

(20, 20, 8, 8)
(20, 20, 4, 8)
0 Omega_I: 0.5885816431929312
1 Omega_I: 0.588581643192929
2 Omega_I: 0.5885816431929303
3 Omega_I: 0.5885816431929287
4 Omega_I: 0.5885816431929312
5 Omega_I: 0.5885816431929308
6 Omega_I: 0.5885816431929283
7 Omega_I: 0.5885816431929288
8 Omega_I: 0.5885816431929292
9 Omega_I: 0.5885816431929245
10 Omega_I: 0.5885816431929259
11 Omega_I: 0.5885816431929278
12 Omega_I: 0.5885816431929285
13 Omega_I: 0.5885816431929355
14 Omega_I: 0.5885816431929228
15 Omega_I: 0.5885816431929116
16 Omega_I: 0.5885816431929017
17 Omega_I: 0.5885816431928946
18 Omega_I: 0.5885816431928889
19 Omega_I: 0.5885816431928834
20 Omega_I: 0.5885816431928784
21 Omega_I: 0.5885816431928746
22 Omega_I: 0.5885816431928709
23 Omega_I: 0.5885816431928677
24 Omega_I: 0.5885816431928651
25 Omega_I: 0.5885816431928628
26 Omega_I: 0.5885816431928601
27 Omega_I: 0.5885816431928584
28 Omega_I: 0.5885816431928567
29 Omega_I: 0.5885816431928549
30 Omega_I: 0.5885816431928534
31 Omega_I: 0.588581

Timer unit: 1e-09 s

Total time: 2.79153 s
File: /Users/treycole/Repos/WanPy/profiling/../wanpy/wpythtb.py
Function: subspace_selec at line 2132

Line #      Hits         Time  Per Hit   % Time  Line Contents
  2132                                               def subspace_selec(
  2133                                                   self, outer_window="occupied", inner_window=None, twfs=None,
  2134                                                   N_wfs=None, iter_num=1000, tol=1e-5, alpha=1, verbose=False
  2135                                               ):
  2136                                                   # if we haven't done single-shot projection yet (set tilde states)
  2137         1       2000.0   2000.0      0.0          if twfs is not None:
  2138                                                       # initialize tilde states
  2139                                                       twfs = self.get_trial_wfs(twfs)
  2140                                       

In [16]:
%lprun -f WFs.find_optimal_subspace_bands WFs.find_optimal_subspace_bands(verbose=True, iter_num=100, tol=1e-15)

(20, 20, 8, 8)
(20, 20, 4, 8)
0 Omega_I: 0.5885816431929268
1 Omega_I: 0.5885816431929269
2 Omega_I: 0.5885816431929312
3 Omega_I: 0.5885816431929296
4 Omega_I: 0.5885816431929285
5 Omega_I: 0.5885816431929302
6 Omega_I: 0.5885816431929274
7 Omega_I: 0.5885816431929299
8 Omega_I: 0.5885816431929335
9 Omega_I: 0.5885816431929309
10 Omega_I: 0.5885816431929305
11 Omega_I: 0.5885816431929308
12 Omega_I: 0.5885816431929314
13 Omega_I: 0.588581643192932
14 Omega_I: 0.5885816431929377
15 Omega_I: 0.5885816431929244
16 Omega_I: 0.5885816431929131
17 Omega_I: 0.5885816431929042
18 Omega_I: 0.5885816431928969
19 Omega_I: 0.5885816431928909
20 Omega_I: 0.5885816431928856
21 Omega_I: 0.5885816431928814
22 Omega_I: 0.588581643192877
23 Omega_I: 0.5885816431928729
24 Omega_I: 0.5885816431928699
25 Omega_I: 0.5885816431928667
26 Omega_I: 0.588581643192865
27 Omega_I: 0.5885816431928618
28 Omega_I: 0.5885816431928598
29 Omega_I: 0.5885816431928574
30 Omega_I: 0.5885816431928553
31 Omega_I: 0.58858164

Timer unit: 1e-09 s

Total time: 2.51507 s
File: /Users/treycole/Repos/WanPy/profiling/../wanpy/wpythtb.py
Function: find_optimal_subspace_bands at line 1922

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1922                                               def find_optimal_subspace_bands(
  1923                                                   self, N_wfs=None, inner_bands=None, outer_bands="occupied", 
  1924                                                   iter_num=100, verbose=False, tol=1e-10, alpha=1
  1925                                               ):
  1926                                                   """Finds the subspaces throughout the BZ that minimizes the gauge-independent spread. 
  1927                                           
  1928                                                   Used when the inner and outer windows correspond to bands rather than energy values. This function
  1929                                                   is fas

In [17]:
u_wfs_Wan = WFs.tilde_states.get_states()["Cell periodic"]

In [19]:
spread, expc_rsq, expc_r_sq = WFs.spread_recip(decomp=True)

print("After first projection")
print(rf"Spread = {spread[0]}")
print(rf"Omega_I = {spread[1]}")
print(rf"Omega_til = {spread[2]}")

After first projection
Spread = [229.59081713 257.91024282 236.3004609  252.51582131]
Omega_I = 0.5885816431929811
Omega_til = 975.7287605189323


In [4]:
%lprun -f WFs.find_min_unitary WFs.find_min_unitary(eps=1e-3, iter_num=1000, verbose=True, tol=1e-17)

ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (20,20,6,4,4)->(20,20,6,newaxis,newaxis) (6,20,20,4,4)->(6,20,20,newaxis,newaxis)  and requested shape (4,4)

In [16]:
%lprun -f find_optimal_subspace2 find_optimal_subspace2(lat_vecs, orbs, outer_states, u_tilde_wan, iter_num=100, verbose=True)

ValueError: 'shift' and 'axis' should be scalars or 1D sequences

In [15]:
%lprun -f find_optimal_subspace find_optimal_subspace(lat_vecs, orbs, outer_states, u_tilde_wan, iter_num=100, verbose=True)

0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: 0.5114317051940603
4 Omega_I: 0.5079889673197662
5 Omega_I: 0.504598731828422
6 Omega_I: 0.5012619108799339
7 Omega_I: 0.49798763126911627
8 Omega_I: 0.4947889209823177
9 Omega_I: 0.4916798757270802
10 Omega_I: 0.48867378673602585
11 Omega_I: 0.48578198905465403
12 Omega_I: 0.48301327418404705
13 Omega_I: 0.4803737222134296
14 Omega_I: 0.47786681964443983
15 Omega_I: 0.475493746569461
16 Omega_I: 0.4732537429670067
17 Omega_I: 0.4711444912243334
18 Omega_I: 0.46916247631807456
19 Omega_I: 0.46730330365649536
20 Omega_I: 0.4655619672654045
21 Omega_I: 0.46393306868567136
22 Omega_I: 0.4624109910153313
23 Omega_I: 0.46099003422475665
24 Omega_I: 0.45966451817546417
25 Omega_I: 0.45842885934531563
26 Omega_I: 0.45727762651902565
27 Omega_I: 0.45620557987733384
28 Omega_I: 0.4552076971311633
29 Omega_I: 0.4542791896506486
30 Omega_I: 0.45341551094840576
31 Omega_I: 0.45261235938839023
32 Om

Timer unit: 1e-09 s

Total time: 1.09909 s
File: /Users/treycole/Codes/WanPy/WanPy/WanPy.py
Function: find_optimal_subspace at line 657

Line #      Hits         Time  Per Hit   % Time  Line Contents
   657                                           def find_optimal_subspace(
   658                                               lat_vecs, orbs, outer_states, inner_states, iter_num=100, verbose=False, tol=1e-17, alpha=1
   659                                           ):
   660         1      10000.0  10000.0      0.0      if isinstance(inner_states, wf_array):
   661                                                   shape = inner_states._wfs.shape  # [*nks, idx, orb]
   662                                                   inner_states = np.array(inner_states._wfs)
   663                                               else:
   664         1      13000.0  13000.0      0.0          shape = inner_states.shape  # [*nks, idx, orb]
   665                                           
   666       

In [13]:
%timeit find_min_unitary(u_tilde_wan, lat_vecs, orbs, eps=1e-3, iter_num=10, verbose=True, tol=1e-17)

0 Omega_til = 0.1549140205024412, Grad mag: 4.726324345017295
1 Omega_til = 0.15480909872956716, Grad mag: 4.7002967106233084
2 Omega_til = 0.15471152306817362, Grad mag: 4.674373024957905
3 Omega_til = 0.15462057378786062, Grad mag: 4.648555139703777
4 Omega_til = 0.1545356140277706, Grad mag: 4.622844737455422
5 Omega_til = 0.15445607939209788, Grad mag: 4.59724334487224
6 Omega_til = 0.1543814689220185, Grad mag: 4.571752344646317
7 Omega_til = 0.15431133725559754, Grad mag: 4.546372986410804
8 Omega_til = 0.1542452878134296, Grad mag: 4.521106396698661
9 Omega_til = 0.15418296687108415, Grad mag: 4.495953588049222
0 Omega_til = 0.1549140205024412, Grad mag: 4.726324345017295
1 Omega_til = 0.15480909872956716, Grad mag: 4.7002967106233084
2 Omega_til = 0.15471152306817362, Grad mag: 4.674373024957905
3 Omega_til = 0.15462057378786062, Grad mag: 4.648555139703777
4 Omega_til = 0.1545356140277706, Grad mag: 4.622844737455422
5 Omega_til = 0.15445607939209788, Grad mag: 4.5972433448722

In [15]:
%timeit find_min_unitary(u_tilde_wan, lat_vecs, orbs, eps=1e-3, iter_num=10, verbose=True, tol=1e-17)

0 Omega_til = 0.1549140205024412, Grad mag: 4.726324345017295
1 Omega_til = 0.15480909872956716, Grad mag: 4.7002967106233084
2 Omega_til = 0.15471152306817362, Grad mag: 4.674373024957905
3 Omega_til = 0.15462057378786062, Grad mag: 4.648555139703777
4 Omega_til = 0.1545356140277706, Grad mag: 4.622844737455422
5 Omega_til = 0.15445607939209788, Grad mag: 4.59724334487224
6 Omega_til = 0.1543814689220185, Grad mag: 4.571752344646317
7 Omega_til = 0.15431133725559754, Grad mag: 4.546372986410804
8 Omega_til = 0.1542452878134296, Grad mag: 4.521106396698661
9 Omega_til = 0.15418296687108415, Grad mag: 4.495953588049222
0 Omega_til = 0.1549140205024412, Grad mag: 4.726324345017295
1 Omega_til = 0.15480909872956716, Grad mag: 4.7002967106233084
2 Omega_til = 0.15471152306817362, Grad mag: 4.674373024957905
3 Omega_til = 0.15462057378786062, Grad mag: 4.648555139703777
4 Omega_til = 0.1545356140277706, Grad mag: 4.622844737455422
5 Omega_til = 0.15445607939209788, Grad mag: 4.5972433448722

In [19]:
%timeit find_optimal_subspace(lat_vecs, orbs, outer_states, u_tilde_wan, iter_num=10, verbose=True)

0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: 0.5114317051940603
4 Omega_I: 0.5079889673197662
5 Omega_I: 0.504598731828422
6 Omega_I: 0.5012619108799339
7 Omega_I: 0.49798763126911627
8 Omega_I: 0.4947889209823177
9 Omega_I: 0.4916798757270802
0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: 0.5114317051940603
4 Omega_I: 0.5079889673197662
5 Omega_I: 0.504598731828422
6 Omega_I: 0.5012619108799339
7 Omega_I: 0.49798763126911627
8 Omega_I: 0.4947889209823177
9 Omega_I: 0.4916798757270802
0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: 0.5114317051940603
4 Omega_I: 0.5079889673197662
5 Omega_I: 0.504598731828422
6 Omega_I: 0.5012619108799339
7 Omega_I: 0.49798763126911627
8 Omega_I: 0.4947889209823177
9 Omega_I: 0.4916798757270802
0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: